# Helgeson, Delany, Nesbitt and Bird Standard State Code Generator 
## Special case for Quartz

Helgeson HC, Delany JM, Nesbitt HW, and Bird DK (1978) Summary and critique of the thermodynamic properties of rock-forming minerals, American Journal of Sciences, 278-A, 229pp

This notebook generates code to calculate thermodynamic properties of the mineral quartz, which is treated as a special case in Helgeson et al. (1992). The general case for minerals and gasses is provided in [another notebook](HDNB.ipynb).  Code generation for teh properties of aqueous species is treated in yet [another notebook](HKF.ipynb).

Required system packages and initialization

In [ ]:
import pandas as pd
import numpy as np
import sympy as sym
import pickle
sym.init_printing()

Required ENKI packages

In [ ]:
from thermoengine import coder

# Formulation
There are three classes of terms:
1. Terms that apply over the whole of $T$-, $P$-space, $T_r \le T$, $P_r \le P$
2. Terms that apply over a specified range of $T$-, $P$-space, $(T_{r_\lambda},P_{r_\lambda}) \le (T,P) \le (T_\lambda,P_\lambda)$
3. Terms that apply to a specific $T_t$ and $P_t$ and higher $T$, $P$, $T_t \le T$, $P_t \le P$

### Helgeson et al. (1978) model structure
The isobaric heat capacity is represented by a Maier-Kelley type expression, ${C_P} = a + bT + \frac{c}{{{T^2}}}$. A phase transition is represented by a finite change in enthalpy and volume (and since the free energy change at the transition is zero, a dependent change in entropy) at some $T_t$. Above the transition temperature, which can be pressure dependent, a new heat capacity expression is used.  There can be up to three transitions for a single phase.  Gasses and most minerals do not undergo phase transition.  MOre than one phase transition is unusual.

Create a model class for the Gibbs free energy

In [ ]:
model = coder.StdStateModel()

Retrieve sympy symbols for model variables and reference conditions

In [ ]:
T = model.get_symbol_for_t()
P = model.get_symbol_for_p()
Tr = model.get_symbol_for_tr()
Pr = model.get_symbol_for_pr()

- The isobaric heat capacity terms parameterized as: $C_P = a_0 + b_0 T + c_0 / T^2$
- Third law entropy: $ S_{Tr,Pr} $
- Enthalpy of formation from the elements, $ \Delta H_{Tr,Pr} $
- $T_t$ is the phase transition temperature
- $\Delta H_t$ is the transition enthalpy at $T_t$
- $\Delta V_t$ is the transition volume at $T_t$
- ${\left. {\frac{{dP}}{{dT}}} \right|_t}$ is the Clapyron slope of the phase transition at $T_t$.  Here the inverse is used in order to more easily parameterize the model
- In this special case, the volume of quartz in the low symmetry polymorph is taken to be a function of both temperature and pressure (see Eq. 109 in Helgeson et al. (1978), page 83)

In [ ]:
a0,b0,c0,a1,b1,c1 = sym.symbols('a0 b0 c0 a1 b1 c1')
Tt = sym.symbols('Tt')
DeltaH = sym.symbols('DeltaH')
DeltaV = sym.symbols('DeltaV')
dPdT = sym.symbols('dPdT')
VTtPta = sym.symbols('VTtPta')
dVdPa = sym.symbols('dVdPa')
STrPr,GTrPr,VTrPr = sym.symbols('S_TrPr G_TrPr V_TrPr')

Contributions applicable over the whole of *T*, *P* space ...

In [ ]:
g = GTrPr - STrPr*(T-Tr)
params = [('G_TrPr','J/m',GTrPr), ('S_TrPr','J/K-m',STrPr)]
model.add_expression_to_model(g, params)

Heat capacity contribution up to the $\alpha$-$\beta$ transition ...

In [ ]:
Cp = a0+b0*T+c0/T**2
g = sym.integrate(Cp,(T,Tr,T)) - T*sym.integrate(Cp/T,(T,Tr,T))
params = [('a0','J/K-m',a0), ('b0','J/K^2-m',b0), ('c0','J-K/m',c0), ('Tt','K',Tt)]
model.add_expression_to_model(g, params, exp_type='restricted', lower_limits=(None,None), upper_limits=(Tt,None))
gAtTt = g.subs(T, Tt)

Volume integral up to the $\alpha$-$\beta$ transition

In [ ]:
a_alpha = Tt - Pr/dPdT - Tr
b_alpha = VTtPta - VTrPr + dVdPa*Pr
g = VTrPr*(P-Pr) - dVdPa*(2*Pr*(P-Pr)-(P*P-Pr*Pr))/2 - dVdPa*dPdT*(T-Tr)*(P-Pr) + dPdT*(b_alpha+a_alpha*dVdPa*dPdT)*(T-Tr)*(
    sym.ln((a_alpha+P/dPdT)/(a_alpha+Pr/dPdT)))
params = [('V_TrPr', 'J/bar-m', VTrPr), ('VTtPta','J/bar-m',VTtPta), ('dVdPa','J/bar^2-m',dVdPa), ('dPdT','bar/K',dPdT)]
model.add_expression_to_model(g, params, exp_type='restricted', lower_limits=(None,None), upper_limits=(Tt,None))

First order phase transition contribution at the $\alpha$-$\beta$ transition ...

In [ ]:
g = -(T-Tt)*DeltaH/Tt
params = [('DeltaH','J/m',DeltaH)]
model.add_expression_to_model(g, params, exp_type='restricted', lower_limits=(Tt,None), upper_limits=(None,None))

Volume integral above the $\alpha$-$\beta$ transition

In [ ]:
Pt = Pr + dPdT*(T-Tt)
dVdTa = (VTtPta-VTrPr-dVdPa*(Pt-Pr))/(Tt-Tr)
g = VTrPr*(P-Pt) + (VTtPta+DeltaV)*(Pt-Pr) - dVdPa*(2*Pr*(P-Pt)-(P*P-Pt*Pt))/2 - dVdPa*dPdT*(T-Tr)*(P-Pt) + dPdT*(
    b_alpha+a_alpha*dVdPa*dPdT)*(T-Tr)*(sym.ln((a_alpha+P/dPdT)/(a_alpha+Pt/dPdT)))
params = [('DeltaV','J/bar-m',DeltaV)]
model.add_expression_to_model(g, params, exp_type='restricted', lower_limits=(Tt,None), upper_limits=(None,None))

Heat capacity contribution above the $\alpha$-$\beta$ transition ...

In [ ]:
Cp = a1+b1*T+c1/T**2
g = sym.integrate(Cp,(T,Tt,T)) - T*sym.integrate(Cp/T,(T,Tt,T))
params = [('a1','J/K-m',a1), ('b1','J/K^2-m',b1), ('c1','J-K/m',c1)]
model.add_expression_to_model(g+gAtTt, params, exp_type='restricted', lower_limits=(Tt,None), upper_limits=(None,None))

## Code Print the Model, compile the code and link a Python module
Name the model class

In [ ]:
model.set_module_name('hdnb')

### Retrieve a phase from the Helgeson et al. (1978) database ...
Load a SLOP file, which consists of dictionaries of Pandas dataframes, and extract the parameters of a phase from the one of the dataframes.  

In [ ]:
with open('slop16_v3_1.dict', 'rb') as pickle_file:
    slop_d = pickle.load(pickle_file)

Contents of the SLOP dictionary ...

In [ ]:
for key in slop_d.keys():
    print ('key:', key)

Select the Pandas dataframe for minerals with a single phase transition and retrieve parameters for quartz ...

In [ ]:
min_df = slop_d.get("mineral_one_transition")
labels = min_df.columns.tolist()
content = min_df.loc[min_df['Name'] == 'Quartz'].get_values().tolist()[0]
quartz = dict(zip(labels,content))
for key in quartz.keys():
    print ("{0:<25.25s} {1:15.15s}".format(key, quartz[key]))

Load a parameter dictionary with SLOP values ...

In [ ]:
calToJoules = 4.184
param_dict = {
    'S_TrPr':float(quartz['S (cal/K-m)'])*calToJoules,
    'G_TrPr':float(quartz['deltaG (cal/m)'])*calToJoules,
    'V_TrPr':float(quartz['V (cc/m)'])/10.0, # J/bar-m
    'a0':float(quartz['a (cal/K-m)'])*calToJoules,
    'b0':float(quartz['b (10^3 cal/K^2-m)'])*calToJoules/1000.0,
    'c0':float(quartz['c (10^-5 cal-K/m)'])*calToJoules*100000.0,
    'Tt':float(quartz['Tt1 (K)']),
    'DeltaH':float(quartz['DeltaHt1 (cal/m)'])*calToJoules,
    'DeltaV':float(quartz['deltaVt1 (cc/m)'])/10.0, # J/bar-m
    'dPdT':float(quartz['dPdTt1 (bar/K)']),
    'VTtPta': 23.348/10.0, # J/bar-m (from Helgeson et al., 1978)
    'dVdPa':-4.973e-5/10.0,  # J/bar^2-m (from Helgeson et al., 1978)
    'a1':float(quartz['at1 (cal/K-m)'])*calToJoules,
    'b1':float(quartz['bt1 (10^3 cal/K^2-m)'])*calToJoules/1000.0,
    'c1':float(quartz['ct1 (10^-5 cal-K/m)'])*calToJoules*100000.0,
    'T_r':298.15,
    'P_r':1.0
}
phase_name = quartz['Name']
formula = quartz['Formula']

Make a working sub-directory and move down into the directory.  This is done so that generated files will not clash between alternate model configurations.

In [ ]:
model_working_dir = "hdnb"
!mkdir -p {model_working_dir}
%cd {model_working_dir}

Note that the call to
```
model.create_code_module(phase=phase_name, formula=formula, params=param_dict)
```
generates fast code with unmodifiable model parameters and "calibration-" related functions.  The call to:
```
model.create_code_module(phase=phase_name, formula=formula, params=param_dict, module_type='calib')
```
generates code suitable for model parameter calibration.  
model_type is "fast" or "calib"

In [ ]:
model_type = "fast"

In [ ]:
result = model.create_code_module(phase=phase_name, formula=formula, params=param_dict, module_type=model_type)

## Import the new module and test the model

In [ ]:
import hdnb
%cd ..

### Available in both "Fast" and "Calib" code versions 
Execute the "fast" or "calibration" code metadata retrieval functions:

In [ ]:
try:
    print(hdnb.cy_Quartz_hdnb_identifier())
    print(hdnb.cy_Quartz_hdnb_name())
    print(hdnb.cy_Quartz_hdnb_formula())
    print(hdnb.cy_Quartz_hdnb_mw())
    print(hdnb.cy_Quartz_hdnb_elements())
except AttributeError:
    pass
try:
    print(hdnb.cy_Quartz_hdnb_calib_identifier())
    print(hdnb.cy_Quartz_hdnb_calib_name())
    print(hdnb.cy_Quartz_hdnb_calib_formula())
    print(hdnb.cy_Quartz_hdnb_calib_mw())
    print(hdnb.cy_Quartz_hdnb_calib_elements())
except AttributeError:
    pass

### Supcrt reference properties at 1000 K and 5000 bars  

**Note**: These numbers are from SUPCRT92 (via CHINOSZ).  They are incorrect because Helgeson et al (1992) assumed that $P_{\alpha-\beta}$ is independent of $T$.  It is not. Helgeson et al. (1978) write (Eq 111):  

$\Delta G_{T,P}^o = \Delta G_{T,{P_r}}^o + V_{\alpha ,{P_r},{T_r}}^o\left( {P - P*} \right) + V_{\beta ,{P_r},{T_r}}^o\left( {P - P*} \right) - \frac{{{c_\alpha }\left[ {2{P_r}\left( {P - P*} \right) - \left( {{P^2} - P{*^2}} \right)} \right]}}{2} - {c_\alpha }k\left( {T - {T_r}} \right)\left( {P - P*} \right)$$ + k\left( {{b_\alpha } + {a_\alpha }{c_\alpha }k} \right)\left( {T - {T_r}} \right)\ln \frac{{{a_\alpha } + P/k}}{{{a_\alpha } + P*/k}}$  

where $P*$ is taken to be $P_r$ below the $\alpha-\beta$ phase transition and $P_{\alpha-\beta}$, where ${P_{\alpha-\beta}} = {P_r} + k\left( {T - {T_r}} \right)$, above the transition. In talking the temperature derivative of the above expression, Helgeson et al. (1978) assume that $P*$ is independent of $T$, e.g. their equations 110 and 114.  This eroneous assumption generates incorrect values for the temperature partial derivatives of the Gibbs free energy above the $\alpha-\beta$ transition.  Consequently, the entropy and heat capacity values calculated from SUPCRT92 are in error by about 2% and 8%, respectively.

In [ ]:
ref_properties = {
    'Plow':{'T':1000.0, 'P':1.0, 'G':-218580.1, 'H':-206731.2, 'S':27.79857, 'V':23.72, 'Cp':16.35},
    'Phigh':{'T':1000.0, 'P':5000.0, 'G':-215746, 'H':-203897.1, 'S':27.79857, 'V':23.72, 'Cp':16.35}
}

Evaluate functions at temperature (K) and pressure (bars)

In [ ]:
test_case = 'Phigh'
t = ref_properties[test_case]['T']
p = ref_properties[test_case]['P']

Execute the standard thermodynamic property retrieval functions:

In [ ]:
fmt = "{0:<10.10s} {1:13.6e} {2:<10.10s}"
fmc = "{0:<10.10s} {1:13.6e} {2:13.6e} {3:10.3f} % {4:<10.10s}"
try:
    print(fmc.format('G', hdnb.cy_Quartz_hdnb_g(t,p), 
                     ref_properties[test_case]['G']*4.184, 
                     100.0*(hdnb.cy_Quartz_hdnb_g(t,p)-ref_properties[test_case]['G']*4.184)/
                     ((ref_properties[test_case]['G']*4.184) if ref_properties[test_case]['G'] != 0 else 1.0), 
                     'J/m'))
    print(fmc.format('dGdT', hdnb.cy_Quartz_hdnb_dgdt(t,p), 
                     -ref_properties[test_case]['S']*4.184, 
                     100.0*(hdnb.cy_Quartz_hdnb_dgdt(t,p)+ref_properties[test_case]['S']*4.184)/(-ref_properties[test_case]['S']*4.184), 
                     'J/K-m'))
    print(fmc.format('dGdP', hdnb.cy_Quartz_hdnb_dgdp(t,p), 
                     ref_properties[test_case]['V']/10.0, 
                     100.0*(hdnb.cy_Quartz_hdnb_dgdp(t,p)-ref_properties[test_case]['V']/10.0)/
                     ((ref_properties[test_case]['V']/10.0) if ref_properties[test_case]['V'] != 0.0 else 1.0), 
                     'J/bar-m'))
    print(fmc.format('d2GdT2', hdnb.cy_Quartz_hdnb_d2gdt2(t,p), 
                     -ref_properties[test_case]['Cp']*4.184/t, 
                     100.0*(hdnb.cy_Quartz_hdnb_d2gdt2(t,p)+ref_properties[test_case]['Cp']*4.184/t)/
                     (ref_properties[test_case]['Cp']*4.184/t), 
                     'J/K^2-m'))
    print(fmt.format('d2GdTdP', hdnb.cy_Quartz_hdnb_d2gdtdp(t,p), 'J/K-bar-m'))
    print(fmt.format('d2GdP2', hdnb.cy_Quartz_hdnb_d2gdp2(t,p), 'J/bar^2-m'))
    print(fmt.format('d3GdT3', hdnb.cy_Quartz_hdnb_d3gdt3(t,p), 'J/K^3-m'))
    print(fmt.format('d3GdT2dP', hdnb.cy_Quartz_hdnb_d3gdt2dp(t,p), 'J/K^2-bar-m'))
    print(fmt.format('d3GdTdP2', hdnb.cy_Quartz_hdnb_d3gdtdp2(t,p), 'J/K-bar^2-m'))
    print(fmt.format('d3GdP3', hdnb.cy_Quartz_hdnb_d3gdp3(t,p), 'J/bar^3-m'))
    print(fmc.format('S', hdnb.cy_Quartz_hdnb_s(t,p), 
                     ref_properties[test_case]['S']*4.184, 
                     100.0*(hdnb.cy_Quartz_hdnb_s(t,p)-ref_properties[test_case]['S']*4.184)/(ref_properties[test_case]['S']*4.184), 
                     'J/K-m'))
    print(fmc.format('V', hdnb.cy_Quartz_hdnb_v(t,p), 
                     ref_properties[test_case]['V']/10.0, 
                     100.0*(hdnb.cy_Quartz_hdnb_dgdp(t,p)-ref_properties[test_case]['V']/10.0)/
                     ((ref_properties[test_case]['V']/10.0) if ref_properties[test_case]['V'] != 0.0 else 1.0), 
                     'J/bar-m'))
    print(fmt.format('Cv', hdnb.cy_Quartz_hdnb_cv(t,p), 'J/K-m'))
    print(fmc.format('Cp', hdnb.cy_Quartz_hdnb_cp(t,p), 
                     ref_properties[test_case]['Cp']*4.184, 
                     100.0*(hdnb.cy_Quartz_hdnb_cp(t,p)-ref_properties[test_case]['Cp']*4.184)/(ref_properties[test_case]['Cp']*4.184), 
                     'J/K-m'))
    print(fmt.format('dCpdT', hdnb.cy_Quartz_hdnb_dcpdt(t,p), 'J/K^2-m'))
    print(fmt.format('alpha', hdnb.cy_Quartz_hdnb_alpha(t,p), '1/K'))
    print(fmt.format('beta', hdnb.cy_Quartz_hdnb_beta(t,p), '1/bar'))
    print(fmt.format('K', hdnb.cy_Quartz_hdnb_K(t,p), 'bar'))
    print(fmt.format('Kp', hdnb.cy_Quartz_hdnb_Kp(t,p), ''))
except AttributeError:
    pass
try:
    print(fmt.format('G', hdnb.cy_Quartz_hdnb_calib_g(t,p), 'J/m'))
    print(fmt.format('dGdT', hdnb.cy_Quartz_hdnb_calib_dgdt(t,p), 'J/K-m'))
    print(fmt.format('dGdP', hdnb.cy_Quartz_hdnb_calib_dgdp(t,p), 'J/bar-m'))
    print(fmt.format('d2GdP2', hdnb.cy_Quartz_hdnb_calib_d2gdt2(t,p), 'J/K^2-m'))
    print(fmt.format('d2GdTdP', hdnb.cy_Quartz_hdnb_calib_d2gdtdp(t,p), 'J/K-bar-m'))
    print(fmt.format('d2GdP2', hdnb.cy_Quartz_hdnb_calib_d2gdp2(t,p), 'J/bar^2-m'))
    print(fmt.format('d3GdT3', hdnb.cy_Quartz_hdnb_calib_d3gdt3(t,p), 'J/K^3-m'))
    print(fmt.format('d3GdT2dP', hdnb.cy_Quartz_hdnb_calib_d3gdt2dp(t,p), 'J/K^2-bar-m'))
    print(fmt.format('d3GdTdP2', hdnb.cy_Quartz_hdnb_calib_d3gdtdp2(t,p), 'J/K-bar^2-m'))
    print(fmt.format('d3GdP3', hdnb.cy_Quartz_hdnb_calib_d3gdp3(t,p), 'J/bar^3-m'))
    print(fmt.format('S', hdnb.cy_Quartz_hdnb_calib_s(t,p), 'J/K-m'))
    print(fmt.format('V', hdnb.cy_Quartz_hdnb_calib_v(t,p), 'J/bar-m'))
    print(fmt.format('Cv', hdnb.cy_Quartz_hdnb_calib_cv(t,p), 'J/K-m'))
    print(fmt.format('Cp', bhdnb.cy_Quartz_hdnb_calib_cp(t,p), 'J/K-m'))
    print(fmt.format('dCpdT', hdnb.cy_Quartz_hdnb_calib_dcpdt(t,p), 'J/K^2-m'))
    print(fmt.format('alpha', hdnb.cy_Quartz_hdnb_calib_alpha(t,p), '1/K'))
    print(fmt.format('beta', hdnb.cy_Quartz_hdnb_calib_beta(t,p), '1/bar'))
    print(fmt.format('K', hdnb.cy_Quartz_hdnb_calib_K(t,p), 'bar'))
    print(fmt.format('Kp', hdnb.cy_Quartz_hdnb_calib_Kp(t,p), ''))
except AttributeError:
    pass

### Available only in the "Calib" versions of generated code
Execute the parameter value/metadata functions.  
These functions are only defined for the "calibration" model code implementation:

In [ ]:
try:
    np = hdnb.cy_Quartz_hdnb_get_param_number()
    names = hdnb.cy_Quartz_hdnb_get_param_names()
    units = hdnb.cy_Quartz_hdnb_get_param_units()
    values = hdnb.cy_Quartz_hdnb_get_param_values()
    fmt = "{0:<10.10s} {1:13.6e} {2:13.6e} {3:<10.10s}"
    for i in range(0,np):
        print(fmt.format(names[i], values[i], hdnb.cy_Quartz_hdnb_get_param_value(i), units[i]))
except AttributeError:
    pass

Test the functions that allow modification of the array of parameter values

In [ ]:
try:
    values[1] = 100.0
    hdnb.cy_Quartz_hdnb_set_param_values(values)
    fmt = "{0:<10.10s} {1:13.6e} {2:13.6e} {3:<10.10s}"
    for i in range(0,np):
        print(fmt.format(names[i], values[i], hdnb.cy_Quartz_hdnb_get_param_value(i), units[i]))
except (AttributeError, NameError):
    pass

Test the functions that allow modification of a particular parameter value

In [ ]:
try:
    hdnb.cy_Quartz_hdnb_set_param_value(1, 1.0)
    fmt = "{0:<10.10s} {1:13.6e} {2:13.6e} {3:<10.10s}"
    for i in range(0,np):
        print(fmt.format(names[i], values[i], hdnb.cy_Quartz_hdnb_get_param_value(i), units[i]))
except AttributeError:
    pass

Evaluate parameter derivatives ...

In [ ]:
try:
    fmt = "    {0:<10.10s} {1:13.6e}"
    for i in range(0, np):
        print ('Derivative with respect to parameter: ', names[i], ' of')
        print (fmt.format('G', hdnb.cy_Quartz_hdnb_dparam_g(t, p, i)))
        print (fmt.format('dGdT', hdnb.cy_Quartz_hdnb_dparam_dgdt(t, p, i)))
        print (fmt.format('dGdP', hdnb.cy_Quartz_hdnb_dparam_dgdp(t, p, i)))
        print (fmt.format('d2GdT2', hdnb.cy_Quartz_hdnb_dparam_d2gdt2(t, p, i)))
        print (fmt.format('d2GdTdP', hdnb.cy_Quartz_hdnb_dparam_d2gdtdp(t, p, i)))
        print (fmt.format('d2GdP2', hdnb.cy_Quartz_hdnb_dparam_d2gdp2(t, p, i)))
        print (fmt.format('d3GdT3', hdnb.cy_Quartz_hdnb_dparam_d3gdt3(t, p, i)))
        print (fmt.format('d3GdT2dP', hdnb.cy_Quartz_hdnb_dparam_d3gdt2dp(t, p, i)))
        print (fmt.format('d3GdTdP2', hdnb.cy_Quartz_hdnb_dparam_d3gdtdp2(t, p, i)))
        print (fmt.format('d3GdP3', hdnb.cy_Quartz_hdnb_dparam_d3gdp3(t, p, i)))
except (AttributeError, TypeError):
    pass

## Time execution of the code

In [ ]:
try:
    %timeit hdnb.cy_Quartz_hdnb_calib_g(t,p)
except AttributeError:
    pass
try:
    %timeit hdnb.cy_Quartz_hdnb_g(t,p)
except AttributeError:
    pass